In [490]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

In [491]:
indices = pd.read_csv("inflation-indices.csv")
indices.head()

,DATE,USACPIALLMINMEI,USACPIALLMINMEI %,CPIAUCSL,CPIAUCSL %,PCEPILFE,PCEPILFE %
0,1/1/1960,12.361982,NaN,29.37,NaN,17.033,NaN
1,2/1/1960,12.404174,NaN,29.41,NaN,17.065,NaN
2,3/1/1960,12.404174,NaN,29.41,NaN,17.067,NaN
3,4/1/1960,12.446365,NaN,29.54,NaN,17.098,NaN
4,5/1/1960,12.446365,NaN,29.57,NaN,17.125,NaN


In [492]:
items = pd.read_csv("pcecomponents_onlycomponents.csv")
items.head()

,Item
0,Goods
1,Durable goods
2,Motor vehicles and parts
3,New motor vehicles (55)
4,New autos


In [493]:
def removeparen(str):
    i=0
    while i<len(str):
        if(str[i])=="(":
            return str[0:i]
        i+=1
    return str

def numindents(str):
    i=0
    while str[i]==" ":
        i+=1
    return (int)(i/4), removeparen(str[i:])

numindents("    asdf")


(1, 'asdf')

In [494]:
pcecomp = []
for item in items.iterrows():
    indents, string = numindents(item[1].iloc[0])
    pcecomp.append([string, indents])
pcecomp

[['Goods', 0],
 ['Durable goods', 1],
 ['Motor vehicles and parts', 2],
 ['New motor vehicles ', 3],
 ['New autos', 4],
 ['New domestic autos', 5],
 ['New foreign autos', 5],
 ['New light trucks', 4],
 ['Net purchases of used motor vehicles ', 3],
 ['Used autos', 4],
 ['Net transactions in used autos', 5],
 ['Used auto margin', 5],
 ['Employee reimbursement', 5],
 ['Used light trucks', 4],
 ['Net transactions in used trucks', 5],
 ['Used truck margin', 5],
 ['Motor vehicle parts and accessories ', 3],
 ['Tires', 4],
 ['Accessories and parts', 4],
 ['Furnishings and durable household equipment', 2],
 ['Furniture and furnishings ', 3],
 ['Furniture', 4],
 ['Clocks, lamps, lighting fixtures, and other household decorative items', 4],
 ['Carpets and other floor coverings', 4],
 ['Window coverings', 4],
 ['Household appliances ', 3],
 ['Major household appliances', 4],
 ['Small electric household appliances', 4],
 ['Glassware, tableware, and household utensils ', 3],
 ['Dishes and flatware'

In [495]:
def findmax(list):
    max = -1
    for item in list:
        if item[1] > max:
            max = item[1]
    return max

findmax(pcecomp)

7

In [496]:
levels = {}
def groupbylevel(list, level, levels):
    positions = {}
    i=0
    while i < len(list): 
        if(list[i][1]==level):
            pos = i
            group = []
            group.append(list[i])
            if i < len(list)-1:
                j=1
                temp_i = i
                while list[temp_i+j][1]==level:
                    group.append(list[temp_i+j])
                    if(temp_i+j<len(list)-1):
                        j+=1
                        i+=1
                        #print(temp_i+j, len(list)-1)
                    else:
                        #print("broken", temp_i+j)
                        break
            positions[pos] = group
        i+=1
    levels[level] = positions
groupbylevel(pcecomp,4,levels)

In [497]:
levels = {}

for i in range(8):
    groupbylevel(pcecomp,i,levels)

levels

{0: {0: [['Goods', 0]], 146: [['Services', 0]]},
 1: {1: [['Durable goods', 1]],
  68: [['Nondurable goods', 1]],
  147: [['Household consumption expenditures ', 1]],
  338: [['Final consumption expenditures of nonprofit institutions serving households ',
    1]]},
 2: {2: [['Motor vehicles and parts', 2]],
  19: [['Furnishings and durable household equipment', 2]],
  34: [['Recreational goods and vehicles', 2]],
  58: [['Other durable goods', 2]],
  69: [['Food and beverages purchased for off-premises consumption', 2]],
  100: [['Clothing and footwear', 2]],
  109: [['Gasoline and other energy goods', 2]],
  116: [['Other nondurable goods', 2]],
  148: [['Housing and utilities', 2]],
  168: [['Health care', 2]],
  186: [['Transportation services', 2]],
  205: [['Recreation services', 2]],
  230: [['Food services and accommodations', 2]],
  248: [['Financial services and insurance', 2]],
  276: [['Other services', 2]],
  339: [['Gross output of nonprofit institutions ', 2]],
  352: [['

In [498]:
import networkx as nx
from pyvis.network import Network

In [499]:
testlist = [0, 60, 120, 160]

def findfit(number, list):
    fit = -1
    for i in range(len(list)):
        if number >= list[i]:
            fit = list[i]
        else:
            break
    return fit

findfit(61, testlist)
            

60

In [500]:
G = nx.Graph()

for i in range(8):
    colors = {0:"red", 1:"orange", 2:"yellow", 3:"green", 4:"blue", 5:"indigo", 6:"violet", 7:"brown"}
    for item in levels[i]:
        G.add_node(levels[i][item][0][0],size=((10-i)*6), color = colors[i])

for i in range(8):
    if i<7:
        parentlist = list(levels[i].keys())
        for item in levels[i+1]:
            G.add_edge(pcecomp[findfit(item, parentlist)][0], pcecomp[item][0])
            
    

In [501]:
net = Network(notebook = True, height='1000px', width='1000px')
net.from_nx(G)
net.show_buttons(filter_=['nodes','edges','physics'])
net.show("pcenetwork.html")

In [502]:
# https://stackoverflow.com/questions/57512155/how-to-draw-a-tree-more-beautifully-in-networkx
# beautiful trees

In [503]:
import pydot
from networkx.drawing.nx_pydot import graphviz_layout

In [504]:
cpiitems = pd.read_csv('cpicomponents.csv')
cpiitems.head()

,0,All items
0,1,Food
1,2,Food at home
2,3,Cereals and bakery products
3,4,Cereals and cereal products
4,5,Flour and prepared flour mixes


In [505]:
cpicomps = []
for item in cpiitems.iterrows():
    cpicomps.append([removeparen(item[1].iloc[1]), item[1].iloc[0]-1])
cpicomps

[['Food', 0],
 ['Food at home', 1],
 ['Cereals and bakery products', 2],
 ['Cereals and cereal products', 3],
 ['Flour and prepared flour mixes', 4],
 ['Breakfast cereal', 4],
 ['Rice, pasta, cornmeal', 4],
 ['Rice', 5],
 ['Bakery products', 4],
 ['Bread', 5],
 ['White bread', 6],
 ['Bread other than white', 6],
 ['Fresh biscuits, rolls, muffins', 5],
 ['Cakes, cupcakes, and cookies', 5],
 ['Cookies', 6],
 ['Fresh cakes and cupcakes', 6],
 ['Other bakery products', 5],
 ['Fresh sweetrolls, coffeecakes, doughnuts', 6],
 ['Crackers, bread, and cracker products', 6],
 ['Frozen and refrigerated bakery products, pies, tarts, turnovers', 6],
 ['Meats, poultry, fish, and eggs', 2],
 ['Meats, poultry, and fish', 3],
 ['Meats', 4],
 ['Beef and veal', 5],
 ['Uncooked ground beef', 6],
 ['Uncooked beef roasts', 6],
 ['Uncooked beef steaks', 6],
 ['Uncooked other beef and veal', 6],
 ['Pork', 5],
 ['Bacon, breakfast sausage, and related products', 6],
 ['Bacon and related products', 7],
 ['Breakfa

In [506]:
findmax(cpicomps)

7

In [507]:
cpilevels = {}

for i in range(8):
    groupbylevel(cpicomps,i,cpilevels)
cpilevels

{0: {0: [['Food', 0]],
  121: [['Energy', 0]],
  135: [['All items less food and energy', 0]]},
 1: {1: [['Food at home', 1]],
  114: [['Food away from home', 1]],
  122: [['Energy commodities', 1]],
  132: [['Energy services', 1]],
  136: [['Commodities less food and energy commodities', 1]],
  253: [['Services less energy services', 1]]},
 2: {2: [['Cereals and bakery products', 2]],
  20: [['Meats, poultry, fish, and eggs', 2]],
  52: [['Dairy and related products', 2]],
  59: [['Fruits and vegetables', 2]],
  80: [['Nonalcoholic beverages and beverage materials', 2]],
  90: [['Other food at home', 2]],
  115: [['Full service meals and snacks', 2],
   ['Limited service meals and snacks', 2],
   ['Food at employee sites and schools', 2]],
  119: [['Food from vending machines and mobile vendors', 2],
   ['Other food away from home', 2]],
  123: [['Fuel oil and other fuels', 2]],
  126: [['Motor fuel', 2]],
  133: [['Electricity', 2], ['Utility ', 2]],
  137: [['Household furnishings a

In [ ]:
G2 = nx.Graph()

for i in range(8):
    colors = {0:"red", 1:"orange", 2:"yellow", 3:"green", 4:"blue", 5:"indigo", 6:"violet", 7:"brown"}
    for item in cpilevels[i]:
        G2.add_node(cpilevels[i][item][0][0],size=((10-i)*6), color = colors[i])

for i in range(8):
    if i<7:
        parentlist = list(cpilevels[i].keys())
        for item in cpilevels[i+1]:
            G2.add_edge(cpicomps[findfit(item, parentlist)][0], cpicomps[item][0])

net = Network(notebook = True, height='1000px', width='1000px')
net.from_nx(G2)
net.show_buttons(filter_=['nodes','edges','physics'])
net.show("cpinetwork.html")     